In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('..\\..\\camada_bronze\\Crude Oil WTI Futures Historical Data.csv')

df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,12/31/2024,71.25,70.69,71.52,70.43,63.63K,1.02%
1,12/30/2024,70.53,70.00,71.05,69.75,93.10K,0.50%
2,12/27/2024,70.18,69.27,70.32,69.06,79.13K,1.37%
3,12/26/2024,69.23,69.72,70.25,68.94,69.64K,-0.59%
4,12/24/2024,69.64,69.04,69.94,68.95,40.39K,1.21%


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4159 entries, 0 to 4158
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      4159 non-null   object 
 1   Price     4159 non-null   float64
 2   Open      4159 non-null   float64
 3   High      4159 non-null   float64
 4   Low       4159 non-null   float64
 5   Vol.      4029 non-null   object 
 6   Change %  4159 non-null   object 
dtypes: float64(4), object(3)
memory usage: 227.6+ KB


In [4]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,12/31/2024,71.25,70.69,71.52,70.43,63.63K,1.02%
1,12/30/2024,70.53,70.00,71.05,69.75,93.10K,0.50%
2,12/27/2024,70.18,69.27,70.32,69.06,79.13K,1.37%
3,12/26/2024,69.23,69.72,70.25,68.94,69.64K,-0.59%
4,12/24/2024,69.64,69.04,69.94,68.95,40.39K,1.21%


In [5]:
df['Vol.'] = df['Vol.'].str[:-1]
df = df.rename(columns={"Vol.":"Volume"})
df.head()

,Date,Price,Open,High,Low,Volume,Change %
0,12/31/2024,71.25,70.69,71.52,70.43,63.63,1.02%
1,12/30/2024,70.53,70.00,71.05,69.75,93.10,0.50%
2,12/27/2024,70.18,69.27,70.32,69.06,79.13,1.37%
3,12/26/2024,69.23,69.72,70.25,68.94,69.64,-0.59%
4,12/24/2024,69.64,69.04,69.94,68.95,40.39,1.21%


In [6]:
df.isna().sum()

Date          0
Price         0
Open          0
High          0
Low           0
Volume      130
Change %      0
dtype: int64

In [7]:
df["Volume"] = df["Volume"].apply(lambda x: pd.to_numeric(x))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4159 entries, 0 to 4158
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      4159 non-null   object 
 1   Price     4159 non-null   float64
 2   Open      4159 non-null   float64
 3   High      4159 non-null   float64
 4   Low       4159 non-null   float64
 5   Volume    4029 non-null   float64
 6   Change %  4159 non-null   object 
dtypes: float64(5), object(2)
memory usage: 227.6+ KB


In [9]:
list_index = [i for i in df[pd.isna(df["Volume"])].index]

In [10]:
for i in list_index:
    df.loc[i, "Volume"] = df[~pd.isna(df["Volume"])].iloc[i-2:i+2]["Volume"].mean()

In [11]:
df.iloc[84]

Date        09/02/2024
Price            73.78
Open              73.0
High             74.39
Low              72.89
Volume        225.3775
Change %         1.10%
Name: 84, dtype: object

In [12]:
# Converter a coluna 'Date' para o formato correto
df['Date'] = pd.to_datetime(df['Date'])

print("Dados após a modificação:")
df.head()


Dados após a modificação:


,Date,Price,Open,High,Low,Volume,Change %
0,2024-12-31,71.25,70.69,71.52,70.43,63.63,1.02%
1,2024-12-30,70.53,70.00,71.05,69.75,93.10,0.50%
2,2024-12-27,70.18,69.27,70.32,69.06,79.13,1.37%
3,2024-12-26,69.23,69.72,70.25,68.94,69.64,-0.59%
4,2024-12-24,69.64,69.04,69.94,68.95,40.39,1.21%


In [13]:
#Tirando colunas que não serão usadas
df = df.drop(['Open', 'High', 'Low'], axis=1)


In [14]:
display(df)

,Date,Price,Volume,Change %
0,2024-12-31,71.25,63.63,1.02%
1,2024-12-30,70.53,93.10,0.50%
2,2024-12-27,70.18,79.13,1.37%
3,2024-12-26,69.23,69.64,-0.59%
4,2024-12-24,69.64,40.39,1.21%
...,...,...,...,...
4154,2009-01-08,41.70,312.49,-2.18%
4155,2009-01-07,42.63,306.26,-12.25%
4156,2009-01-06,48.58,331.31,-0.47%
4157,2009-01-05,48.81,258.76,5.33%


In [15]:
df.isna().sum()

Date        0
Price       0
Volume      0
Change %    0
dtype: int64

In [16]:
# Salvar o DataFrame modificado
modified_file_path = '..\\..\\camada_prata\\Crude_Oil_clean.csv'
df.to_csv(modified_file_path, index=False)